# LEAPD Demo (Synthetic)

This notebook creates a tiny synthetic EEG-like dataset, saves it to `.mat`, runs a minimal train/test using the Python pipeline, and shows ROC / channel heatmap.

In [ ]:
import sys, os, numpy as np
sys.path.append('src')
from leapd import load_data, build_hyperplanes, compute_leapd_scores
from leapd import evaluate_classification, plot_roc_curve, plot_channel_heatmap
from scipy.io import savemat

# --- 1) Generate synthetic dataset ---
np.random.seed(0)
Fs = 500
n_sub_g1, n_sub_g2 = 8, 8
n_ch = 2
def synth_signal(n=2000, ar=[0.9, -0.3]):
    # AR(2): x[t] = ar1*x[t-1] + ar2*x[t-2] + noise
    x = np.zeros(n)
    noise = np.random.randn(n)*0.5
    for t in range(2, n):
        x[t] = ar[0]*x[t-1] + ar[1]*x[t-2] + noise[t]
    return x

# group1 has slightly different AR than group2
EEG = []
for ch in range(n_ch):
    g1 = [synth_signal(ar=[0.85, -0.25]) for _ in range(n_sub_g1)]
    g2 = [synth_signal(ar=[0.75, -0.15]) for _ in range(n_sub_g2)]
    EEG.append([g1, g2])
Channel_location = [f'CH{c+1}' for c in range(n_ch)]
Filenames = type('F', (), {})()
Filenames.group1 = np.array([f'S{c+1}' for c in range(n_sub_g1)], dtype=object)
Filenames.group2 = np.array([f'S{c+1+n_sub_g1}' for c in range(n_sub_g2)], dtype=object)

mat_path = 'data/EEG_synth.mat'
os.makedirs('data', exist_ok=True)
savemat(mat_path, {'EEG': np.array(EEG, dtype=object), 'Channel_location': np.array(Channel_location, dtype=object), 'Filenames': Filenames}, do_compression=True)
mat_path

In [ ]:
# --- 2) Quick in-memory check of loader shapes ---
from leapd import load_data
DataMap, ChLoc, SubjectIDs, GroupNames = load_data('data/EEG_synth.mat', [], [])
len(DataMap), ChLoc, list(SubjectIDs.keys())[:2]

In [ ]:
# --- 3) Minimal LEAPD computation on one channel: scores + ROC ---
from leapd import filter_data
from leapd.signal import compute_lpc_burg
ch = list(DataMap.keys())[0]
chData = DataMap[ch]
AllData = chData['group1'] + chData['group2']
labels = np.r_[np.ones(len(chData['group1'])), np.zeros(len(chData['group2']))]

Filt = np.array([[0, 0.5, 0], [20, np.inf, 0]], dtype=float)
Xf = filter_data(AllData, Filt, Fs)
order = 8; dim = 3
LPC_all = np.vstack([compute_lpc_burg(x, order)[None, :] for x in Xf])
n1 = len(chData['group1'])
P0, m0 = build_hyperplanes(LPC_all[n1:, :], dim)
P1, m1 = build_hyperplanes(LPC_all[:n1, :], dim)
scores = compute_leapd_scores(LPC_all, P0, m0, P1, m1)
metrics = evaluate_classification(scores, labels)
metrics

In [ ]:
# --- 4) ROC plot ---
plot_roc_curve(labels, scores, title=f'ROC {ch}', save_path='figures/roc_demo.png')
print('Saved ROC to figures/roc_demo.png')

In [ ]:
# --- 5) Channel heatmap of single-channel ACC (dummy values) ---
channels = list(DataMap.keys())
vals = np.random.rand(len(channels))
plot_channel_heatmap(channels, vals, title='Channel Metric Heatmap (Demo)', save_path='figures/heatmap_demo.png')
print('Saved heatmap to figures/heatmap_demo.png')